[587. Erect the Fence](https://leetcode.com/problems/erect-the-fence/)

In [32]:
// using System.Drawing;
// using System.IO;

public class Solution
{
    // private static int _i = 0;

    private bool LineLineIntersection(int[] ap0, int[] ap1, int[] bp0, int[] bp1, out double[] res)
    {
        double a1 = ap1[1] - ap0[1];
        double b1 = ap0[0] - ap1[0];
        double c1 = a1 * ap0[0] + b1 * ap0[1];
 
        double a2 = bp1[1] - bp0[1];
        double b2 = bp0[0] - bp1[0];
        double c2 = a2 * bp0[0] + b2 * bp0[1];
 
        double determinant = a1 * b2 - a2 * b1;
 
        if (determinant == 0)
        {
            res = new double[] { double.MaxValue, double.MaxValue };
            return false;
        }

        double x = (b2 * c1 - b1 * c2) / determinant;
        double y = (a1 * c2 - a2 * c1) / determinant;
        res = new double[] { x, y };
        return true;
    }

    private bool IsPointOnLines(double[] point, int[] a, int[] b, int[] c, int[] d)
    {
        var minX1 = Math.Min(a[0], b[0]);
        var maxX1 = Math.Max(a[0], b[0]);
        var minY1 = Math.Min(a[1], b[1]);
        var maxY1 = Math.Max(a[1], b[1]);
        var minX2 = Math.Min(c[0], d[0]);
        var maxX2 = Math.Max(c[0], d[0]);
        var minY2 = Math.Min(c[1], d[1]);
        var maxY2 = Math.Max(c[1], d[1]);

        var onLine1 = ((minX1 < point[0] && point[0] < maxX1) && (minY1 <= point[1] && point[1] <= maxY1)) ||
            ((minX1 <= point[0] && point[0] <= maxX1) && (minY1 < point[1] && point[1] < maxY1));
        var onPoint1 = (a[0] == point[0] && a[1] == point[1]) || (b[0] == point[0] && b[1] == point[1]);

        var onLine2 = ((minX2 < point[0] && point[0] < maxX2) && (minY2 <= point[1] && point[1] <= maxY2)) ||
            ((minX2 <= point[0] && point[0] <= maxX2) && (minY2 < point[1] && point[1] < maxY2));
        var onPoint2 = (c[0] == point[0] && c[1] == point[1]) || (d[0] == point[0] && d[1] == point[1]);

        return (onLine1 && (onLine2 || onPoint2)) || (onLine2 && (onLine1 || onPoint1));
    }
    
    public int[][] OuterTrees(int[][] trees)
    {
        var shape = new List<int[]>();
        foreach(var tree in trees)
        {
            shape.Add(tree);
            if(shape.Count < 4)
                continue;

            var reset = true;
            while(reset)
            {
                reset = false;
                for(int i = 0; i < shape.Count - 1; i++)
                {
                    for(int j = i + 1; j < shape.Count; j++)
                    {
                        int indexA = i;
                        int indexB = (i + 1) % shape.Count;
                        int indexC = j;
                        int indexD = (j + 1) % shape.Count;
    
                        var success = LineLineIntersection(shape[indexA], shape[indexB], shape[indexC], shape[indexD], out var intersection);
                        // Render(trees,
                        //     shape,
                        //     new()
                        //     {
                        //         new[] { shape[indexA][0], shape[indexA][1], shape[indexB][0], shape[indexB][1] },
                        //         new[] { shape[indexC][0], shape[indexC][1], shape[indexD][0], shape[indexD][1]}
                        //     });
    
                        if(!success)
                            continue;
                        if(!IsPointOnLines(intersection, shape[indexA], shape[indexB], shape[indexC], shape[indexD]))
                            continue;
                        
                        // Render(trees,
                        //     shape,
                        //     new()
                        //     {
                        //         new[] { shape[indexA][0], shape[indexA][1], shape[indexB][0], shape[indexB][1] },
                        //         new[] { shape[indexC][0], shape[indexC][1], shape[indexD][0], shape[indexD][1]}
                        //     },
                        //     intersection);
                        
                        if(Math.Abs(indexB - indexC) < Math.Abs(indexA - indexD))
                            (shape[indexB], shape[indexC]) = (shape[indexC], shape[indexB]);
                        else
                            (shape[indexA], shape[indexD]) = (shape[indexD], shape[indexA]);
                        reset = true;
                    }
                }
            }

            var sumOfEdges = 0;
            for(int i = 0; i < shape.Count; i++)
                sumOfEdges += (shape[(i + 1) % shape.Count][0] - shape[i][0]) * (shape[(i + 1) % shape.Count][1] + shape[i][1]);
            var isClockwise = sumOfEdges > 0;
            
            // Render(trees, shape);

            while(true)
            {
                var shapeChanged = false;
                for(int i = 0; i < shape.Count; i++)
                {
                    var pointA = shape[i];
                    var pointB = shape[(i + 1) % shape.Count];
                    var pointC = shape[(i + 2) % shape.Count];
                    
                    // Render(trees,
                    //     shape,
                    //     new()
                    //     {
                    //         new[] { pointA[0], pointA[1], pointB[0], pointB[1] },
                    //         new[] { pointB[0], pointB[1], pointC[0], pointC[1]}
                    //     },
                    //     new double[] { pointB[0], pointB[1] });
    
                    var vectorA = new double[] { pointA[0] - pointB[0], pointA[1] - pointB[1] };
                    var vectorB = new double[] { pointC[0] - pointB[0], pointC[1] - pointB[1] };
    
                    var crossProd = vectorA[0] * vectorB[1] - vectorA[1] * vectorB[0];
                    if(isClockwise ? crossProd < 0 : crossProd > 0)
                    {
                        // Render(trees,
                        //     shape,
                        //     new()
                        //     {
                        //         new[] { pointA[0], pointA[1], pointB[0], pointB[1] },
                        //         new[] { pointB[0], pointB[1], pointC[0], pointC[1]}
                        //     },
                        //     new double[] { pointB[0], pointB[1] });

                        shape.RemoveAt((i + 1) % shape.Count);
                        i--;
                        shapeChanged = true;
                    }
                }
                if(!shapeChanged)
                    break;
            }
            
            // Render(trees, shape);
        }

        return shape.ToArray();
    }

    // private void Render(int[][] trees, List<int[]> shape, List<int[]> lines = null, double[] point = null)
    // {
    //     int pixelSize = 50;
    //     var directory = "587-erect-the-fence-visualization";
    //     if(!Directory.Exists(directory))
    //         Directory.CreateDirectory(directory);
    //     var maxX = trees.Select(x => x[0]).Max();
    //     var maxY = trees.Select(x => x[1]).Max();
    //     var width = maxX * pixelSize + pixelSize * 3;
    //     var height = maxY * pixelSize + pixelSize * 3;
    //     using var btm = new Bitmap(width, height);
    //     using var g = Graphics.FromImage(btm);
    //     g.FillRectangle(Brushes.Black, 0, 0, width, height);

    //     for(int i = 1; i <= maxX + 2; i++)
    //         g.DrawLine(Pens.DimGray, i * pixelSize, 0, i * pixelSize, height);

    //     for(int i = 1; i <= maxY + 2; i++)
    //         g.DrawLine(Pens.DimGray, 0, i * pixelSize, width, i * pixelSize);

    //     foreach(var pos in trees)
    //         g.FillEllipse(Brushes.Green, pixelSize * 1.25f + pos[0] * pixelSize, pixelSize * 1.25f + pos[1] * pixelSize, pixelSize / 2, pixelSize / 2);

    //     var font = new Font(FontFamily.GenericMonospace, pixelSize / 2);
    //     for(int i = 0; i < shape.Count; i++)
    //     {
    //         g.DrawString($"{i}", font, Brushes.White, shape[i][0] * pixelSize + pixelSize, shape[i][1] * pixelSize + pixelSize);
    //         g.DrawLine(new Pen(Color.Red, 2),
    //             shape[i][0] * pixelSize + pixelSize * 1.5f,
    //             shape[i][1] * pixelSize + pixelSize * 1.5f,
    //             shape[(i + 1) % shape.Count][0] * pixelSize + pixelSize * 1.5f,
    //             shape[(i + 1) % shape.Count][1] * pixelSize + pixelSize * 1.5f);
    //     }

    //     if(point is not null)
    //         g.DrawEllipse(Pens.Cyan, pixelSize * 1.4f + (float)point[0] * pixelSize, pixelSize * 1.4f + (float)point[1] * pixelSize, pixelSize * .2f, pixelSize * .2f);

    //     if(lines is not null)
    //         foreach(var line in lines)
    //             g.DrawLine(new Pen(Color.Cyan, 2),
    //                 line[0] * pixelSize + pixelSize * 1.5f,
    //                 line[1] * pixelSize + pixelSize * 1.5f,
    //                 line[2]  * pixelSize + pixelSize * 1.5f,
    //                 line[3]  * pixelSize + pixelSize * 1.5f);

    //     btm.Save(directory + $"/{_i:0000}.png", System.Drawing.Imaging.ImageFormat.Png);
    //     _i++;
    // }
}

In [33]:
var inputs = new string[]
{
    "[[10,30],[10,32],[10,46],[11,11],[11,22],[11,45],[12,26],[12,28],[12,29],[12,31],[12,47],[13,28],[13,29],[13,31],[13,33],[13,36],[13,41],[13,47],[14,26],[14,45],[15,21],[15,25],[15,33],[15,38],[15,40],[15,45],[16,10],[16,18],[17,12],[17,15],[17,16],[17,22],[17,42],[18,10],[18,24],[18,37],[18,49],[19,10],[19,12],[19,13],[19,41],[19,49],[20,17],[20,19],[20,27],[20,29],[21,31],[22,29],[22,38],[23,15],[23,24],[23,30],[24,12],[25,10],[25,11],[25,14],[25,20],[25,24],[25,45],[25,47],[26,15],[26,16],[26,20],[26,29],[27,15],[27,30],[27,33],[28,25],[29,14],[29,41],[29,46],[30,10],[30,20],[30,28],[30,32],[30,44],[30,48],[31,11],[31,34],[32,13],[32,15],[32,18],[32,35],[32,42],[33,13],[33,20],[33,28],[33,36],[33,45],[34,18],[34,20],[34,22],[34,48],[35,13],[35,27],[35,32],[35,37],[35,41],[35,42],[35,48],[36,12],[36,13],[36,24],[36,40],[36,44],[36,46],[36,47],[37,20],[37,24],[37,26],[37,28],[37,37],[37,49],[38,18],[38,29],[38,36],[38,45],[39,17],[39,23],[39,39],[40,43],[40,47],[40,48],[41,21],[41,30],[41,39],[41,48],[42,17],[42,29],[42,43],[43,10],[43,17],[44,22],[44,24],[44,26],[44,31],[44,37],[44,40],[45,10],[45,35],[45,46],[46,16],[46,30],[46,41],[47,27],[48,28],[48,33],[48,39],[48,48],[49,15],[49,33],[49,34],[49,48],[50,10],[50,27],[50,30],[50,34],[51,10],[51,11],[51,28],[51,32],[52,12],[52,26],[52,32],[53,12],[53,44],[54,12],[54,15],[54,31],[54,43],[54,48],[55,28],[55,34],[55,36],[55,42],[57,15],[57,25],[57,33],[57,42],[58,10],[58,16],[58,32],[58,38],[58,49],[59,10],[59,18],[59,23],[59,27],[59,32],[60,38],[60,45],[61,18],[61,37],[61,38],[61,43],[61,49],[62,14],[62,37],[62,38],[62,41],[62,42],[62,43],[62,46],[63,13],[63,22],[63,25],[63,37],[63,49],[64,31],[65,30],[67,12],[67,15],[67,30],[67,42],[68,12],[68,17],[68,33],[68,36],[69,12],[69,20],[69,22],[69,31],[69,33],[70,28],[70,35],[70,37],[70,40],[70,48],[71,31],[71,40],[72,23],[72,34],[72,35],[72,45],[73,26],[74,10],[74,14],[74,19],[74,24],[75,12],[75,24],[75,31],[76,15],[76,26],[76,31],[76,40],[76,41],[76,45],[77,24],[77,35],[77,42],[78,11],[78,15],[78,23],[78,32],[78,46],[78,48],[80,28],[80,31],[81,11],[81,18],[81,23],[81,25],[81,49],[82,37],[82,38],[82,42],[82,43],[82,45],[82,49],[83,10],[83,17],[83,19],[83,20],[83,22],[83,49],[84,12],[84,14],[84,19],[84,20],[84,30],[84,33],[84,39],[85,15],[85,18],[85,32],[86,14],[86,18],[86,20],[86,26],[86,29],[86,34],[86,47],[86,48],[87,34],[87,36],[87,37],[87,43],[87,49],[88,26],[88,28],[88,48],[89,38],[89,40],[90,13],[90,19],[90,32],[90,34],[90,43],[90,44],[91,10],[91,16],[91,17],[91,27],[91,34],[91,35],[91,48],[92,32],[92,40],[93,26],[93,45],[93,46],[94,10],[94,28],[94,29],[94,42],[94,47],[95,18],[95,20],[95,35],[95,40],[95,43],[96,12],[96,19],[96,27],[96,37],[96,42],[97,12],[97,42],[97,49],[98,20],[98,43],[98,45],[99,12],[99,21],[99,27],[99,34],[99,41],[99,49]]",
    "[[0,1],[0,6],[0,18],[0,28],[0,38],[0,40],[0,46],[0,59],[0,60],[0,70],[0,85],[0,95],[1,6],[1,10],[1,31],[1,39],[1,49],[1,51],[1,55],[1,57],[1,58],[1,60],[1,73],[1,74],[1,82],[1,86],[1,90],[1,99],[2,1],[2,5],[2,15],[2,17],[2,19],[2,21],[2,26],[2,36],[2,46],[2,56],[2,57],[2,66],[2,76],[2,85],[2,89],[2,97],[3,6],[3,19],[3,20],[3,23],[3,54],[3,59],[3,65],[3,99],[4,7],[4,12],[4,20],[4,31],[4,37],[4,39],[4,47],[4,57],[4,61],[4,71],[4,82],[4,87],[4,90],[5,0],[5,17],[5,28],[5,32],[5,46],[5,48],[5,61],[5,63],[5,70],[5,76],[5,79],[5,87],[5,97],[6,3],[6,19],[6,25],[6,31],[6,34],[6,37],[6,39],[6,49],[6,69],[6,75],[6,82],[6,84],[6,88],[7,36],[7,39],[7,48],[7,50],[7,55],[7,81],[7,82],[7,84],[7,89],[8,4],[8,12],[8,18],[8,37],[8,61],[8,70],[8,74],[8,76],[8,80],[8,84],[8,89],[8,93],[8,95],[9,35],[9,50],[9,68],[9,73],[9,75],[9,84],[9,98],[10,3],[10,13],[10,21],[10,24],[10,36],[10,42],[10,47],[10,50],[10,64],[10,88],[10,95],[11,1],[11,3],[11,8],[11,13],[11,24],[11,29],[11,44],[11,45],[11,58],[11,64],[11,85],[11,88],[12,21],[12,26],[12,41],[12,78],[12,87],[12,92],[13,14],[13,17],[13,27],[13,34],[13,37],[13,53],[13,67],[13,76],[13,93],[13,94],[13,96],[14,7],[14,11],[14,15],[14,18],[14,32],[14,64],[14,81],[14,89],[15,1],[15,12],[15,50],[15,53],[15,72],[15,75],[15,96],[16,6],[16,8],[16,13],[16,30],[16,33],[16,34],[16,51],[16,53],[16,56],[16,71],[16,84],[17,9],[17,25],[17,34],[17,55],[17,79],[17,80],[18,0],[18,5],[18,10],[18,17],[18,42],[18,48],[18,50],[18,74],[18,78],[18,85],[18,90],[18,94],[19,13],[19,20],[19,31],[19,65],[19,73],[19,99],[20,1],[20,5],[20,23],[20,41],[20,54],[20,63],[20,66],[20,67],[20,76],[20,84],[21,9],[21,22],[21,32],[21,33],[21,56],[21,70],[21,73],[21,86],[21,87],[22,14],[22,23],[22,25],[22,32],[22,39],[22,41],[22,44],[22,51],[22,52],[22,56],[22,59],[22,70],[22,73],[22,80],[22,81],[22,85],[22,87],[23,4],[23,6],[23,23],[23,24],[23,25],[23,26],[23,34],[23,36],[23,44],[23,58],[23,61],[23,66],[23,69],[23,71],[23,82],[23,84],[23,85],[23,87],[23,91],[24,24],[24,28],[24,44],[24,65],[24,66],[24,69],[24,77],[25,11],[25,41],[25,49],[25,66],[25,69],[25,72],[25,81],[26,22],[26,39],[26,49],[26,55],[26,62],[26,75],[27,22],[27,31],[27,37],[27,39],[27,48],[27,61],[27,70],[27,84],[27,97],[28,14],[28,20],[28,35],[28,42],[28,45],[28,63],[28,66],[28,78],[28,88],[28,91],[29,0],[29,18],[29,23],[29,30],[29,39],[29,48],[29,65],[29,67],[29,68],[29,72],[29,77],[29,81],[29,88],[29,96],[29,98],[30,8],[30,25],[30,35],[30,36],[30,46],[30,62],[30,65],[30,69],[30,78],[30,99],[31,5],[31,9],[31,35],[31,40],[31,75],[31,78],[31,80],[31,85],[31,88],[31,89],[32,16],[32,31],[32,45],[32,53],[32,59],[32,68],[32,73],[32,74],[32,90],[33,11],[33,16],[33,23],[33,26],[33,32],[33,36],[33,37],[33,62],[33,66],[33,67],[33,82],[33,89],[33,92],[33,94],[34,8],[34,13],[34,17],[34,27],[34,29],[34,42],[34,58],[34,60],[34,75],[34,98],[35,6],[35,7],[35,9],[35,17],[35,25],[35,41],[35,50],[35,70],[35,85],[35,87],[35,90],[35,94],[35,98],[36,14],[36,15],[36,26],[36,39],[36,40],[36,69],[36,74],[36,82],[36,84],[36,94],[37,8],[37,48],[37,56],[37,62],[37,63],[37,93],[37,96],[38,20],[38,27],[38,41],[38,42],[38,44],[38,48],[38,49],[38,54],[38,58],[38,64],[38,66],[38,79],[38,81],[38,83],[38,87],[38,92],[39,0],[39,3],[39,9],[39,11],[39,21],[39,34],[39,36],[39,37],[39,51],[39,58],[39,59],[39,70],[39,87],[39,88],[39,91],[39,93],[39,97],[40,6],[40,18],[40,26],[40,31],[40,32],[40,41],[40,45],[40,53],[40,65],[40,70],[41,1],[41,29],[41,63],[41,69],[41,84],[41,91],[41,99],[42,11],[42,12],[42,16],[42,17],[42,21],[42,29],[42,33],[42,79],[42,95],[42,99],[43,2],[43,6],[43,8],[43,10],[43,11],[43,16],[43,17],[43,18],[43,39],[43,55],[43,58],[43,60],[43,70],[43,82],[43,91],[43,94],[44,5],[44,6],[44,26],[44,41],[44,69],[44,71],[44,84],[45,0],[45,13],[45,63],[45,66],[45,67],[45,73],[45,75],[45,93],[46,8],[46,60],[46,90],[46,91],[47,11],[47,38],[47,40],[47,41],[47,42],[47,50],[47,67],[47,75],[47,79],[47,82],[47,92],[48,2],[48,9],[48,28],[48,54],[48,60],[48,76],[48,85],[48,86],[49,27],[49,49],[49,60],[49,62],[49,71],[49,82],[49,85],[49,96],[50,0],[50,21],[50,32],[50,37],[50,39],[50,45],[50,52],[50,59],[50,62],[50,64],[50,67],[50,88],[50,89],[50,90],[50,98],[51,6],[51,8],[51,9],[51,15],[51,17],[51,19],[51,20],[51,21],[51,32],[51,34],[51,39],[51,60],[51,71],[51,73],[51,77],[51,91],[52,0],[52,12],[52,15],[52,26],[52,33],[52,36],[52,48],[52,53],[52,60],[52,78],[52,82],[52,87],[52,91],[53,9],[53,31],[53,40],[53,57],[53,60],[53,67],[53,76],[53,77],[53,81],[53,82],[53,84],[53,89],[53,94],[54,5],[54,16],[54,18],[54,29],[54,33],[54,34],[54,44],[54,49],[54,59],[54,63],[54,82],[54,91],[54,95],[54,96],[55,8],[55,12],[55,27],[55,28],[55,40],[55,80],[55,93],[56,0],[56,3],[56,4],[56,14],[56,23],[56,25],[56,37],[56,45],[56,49],[56,50],[57,7],[57,15],[57,32],[57,45],[57,46],[57,47],[57,48],[57,64],[57,84],[57,92],[57,99],[58,2],[58,22],[58,49],[58,64],[58,74],[58,77],[58,84],[58,92],[59,8],[59,14],[59,30],[59,31],[59,32],[59,39],[59,45],[59,64],[59,65],[59,66],[59,80],[59,87],[60,3],[60,8],[60,19],[60,25],[60,26],[60,36],[60,45],[60,67],[60,76],[60,78],[60,93],[60,97],[61,15],[61,34],[61,44],[61,50],[61,56],[61,57],[61,68],[61,70],[61,93],[61,99],[62,17],[62,21],[62,24],[62,47],[62,63],[62,68],[62,71],[62,79],[62,86],[62,90],[63,3],[63,8],[63,12],[63,18],[63,20],[63,34],[63,36],[63,40],[63,43],[63,51],[63,62],[63,67],[63,74],[63,82],[63,87],[63,90],[64,40],[64,51],[64,65],[64,75],[64,90],[65,2],[65,12],[65,22],[65,31],[65,42],[65,70],[65,83],[65,85],[66,10],[66,19],[66,38],[66,63],[66,81],[66,86],[66,99],[67,29],[67,66],[67,75],[67,76],[67,79],[67,83],[68,20],[68,29],[68,51],[68,57],[68,73],[68,84],[69,15],[69,29],[69,34],[69,37],[69,39],[69,42],[69,51],[69,57],[69,58],[69,59],[69,62],[69,71],[69,74],[69,76],[69,81],[69,82],[70,1],[70,39],[70,40],[70,45],[70,67],[70,85],[71,3],[71,18],[71,20],[71,27],[71,35],[71,46],[71,59],[71,64],[71,70],[71,83],[71,86],[72,14],[72,15],[72,22],[72,24],[72,29],[72,31],[72,34],[72,43],[72,60],[72,73],[72,77],[72,95],[72,99],[73,6],[73,12],[73,26],[73,31],[73,37],[73,40],[73,66],[73,78],[73,83],[73,94],[74,5],[74,14],[74,19],[74,25],[74,45],[74,51],[74,56],[74,81],[74,83],[74,88],[75,14],[75,25],[75,27],[75,36],[75,50],[75,60],[75,82],[75,84],[75,88],[75,92],[76,16],[76,24],[76,25],[76,28],[76,32],[76,42],[76,98],[77,8],[77,21],[77,23],[77,26],[77,30],[77,31],[77,39],[77,40],[77,49],[77,54],[77,56],[77,57],[77,76],[77,89],[77,92],[78,14],[78,20],[78,41],[78,49],[78,50],[78,74],[78,87],[79,11],[79,22],[79,33],[79,38],[79,58],[79,63],[79,64],[79,70],[79,74],[79,91],[79,99],[80,4],[80,7],[80,11],[80,19],[80,33],[80,35],[80,57],[80,58],[80,60],[80,62],[80,77],[80,82],[80,85],[81,5],[81,21],[81,30],[81,34],[81,35],[81,62],[81,63],[81,69],[81,83],[81,91],[81,93],[82,34],[82,37],[82,42],[82,44],[82,51],[82,59],[82,60],[82,73],[83,24],[83,61],[83,66],[83,81],[83,82],[84,14],[84,15],[84,23],[84,25],[84,36],[84,45],[84,54],[84,62],[84,76],[84,79],[84,80],[84,86],[84,95],[84,98],[85,18],[85,28],[85,30],[85,50],[85,61],[85,68],[85,70],[85,93],[85,94],[85,95],[86,6],[86,23],[86,44],[86,47],[86,48],[86,50],[86,51],[86,79],[86,97],[86,98],[87,7],[87,8],[87,16],[87,27],[87,36],[87,46],[87,58],[87,62],[87,67],[87,94],[88,7],[88,25],[88,36],[88,43],[88,45],[88,57],[88,76],[89,3],[89,19],[89,23],[89,24],[89,28],[89,72],[89,81],[90,1],[90,2],[90,6],[90,20],[90,31],[90,35],[90,51],[90,53],[90,54],[90,76],[90,82],[90,83],[90,91],[90,92],[90,94],[91,1],[91,2],[91,21],[91,23],[91,31],[91,40],[91,42],[91,64],[91,65],[91,68],[91,76],[91,85],[92,1],[92,56],[92,75],[92,83],[92,88],[92,91],[93,0],[93,21],[93,51],[93,54],[93,62],[93,72],[93,73],[93,94],[93,96],[93,99],[94,10],[94,21],[94,35],[94,37],[94,39],[94,60],[94,65],[94,67],[94,68],[94,79],[95,8],[95,16],[95,17],[95,24],[95,30],[95,32],[95,41],[96,6],[96,9],[96,61],[96,77],[96,79],[97,5],[97,14],[97,17],[97,21],[97,26],[97,35],[97,40],[97,51],[97,55],[97,72],[97,74],[97,90],[98,2],[98,5],[98,6],[98,10],[98,11],[98,15],[98,16],[98,32],[98,53],[98,54],[98,59],[98,63],[98,65],[98,66],[98,72],[98,77],[98,82],[98,93],[99,6],[99,23],[99,26],[99,29],[99,39],[99,48],[99,58],[99,65],[99,66],[99,86]]",
    "[[0,0],[0,1],[0,2],[1,2],[2,2],[3,2],[3,1],[3,0],[2,0],[1,0],[1,1],[3,3]]",
    "[[0,0],[0,5],[0,9],[2,2],[2,3],[2,4],[3,6],[4,0],[4,1],[4,9],[5,0],[5,1],[5,7],[6,0],[6,2],[6,5],[7,9],[8,1],[8,7],[9,1],[9,2],[9,5]]",
    "[[1,1],[2,2],[2,0],[2,4],[3,3],[4,2]]",
    "[[2,2],[4,2],[3,3],[2,0],[1,1],[2,4]]",
    "[[1,2],[2,2],[4,2]]",
    "[[3,7],[6,8],[7,8],[11,10],[4,3],[8,5],[7,13],[4,13]]",
    "[[0,0],[0,1],[0,2],[1,2],[2,2],[3,2],[3,1],[3,0],[2,0],[1,0],[1,1],[4,3]]",
    "[[9,31],[28,98],[42,54],[37,93],[96,81],[19,45],[30,56],[99,94],[5,76],[24,74],[47,57],[53,95],[44,81],[79,7],[25,6],[5,12],[30,24],[93,7],[94,90],[79,89],[9,13],[8,29],[2,12],[78,19],[64,66],[79,94],[19,67],[92,14],[21,5],[22,2],[78,66],[2,53],[58,95],[86,66],[21,25],[77,61],[75,13],[61,42],[58,85],[69,59],[45,71],[97,97],[49,73],[37,38],[12,2],[26,84],[79,15],[1,75],[37,88],[9,37],[91,33],[57,0],[90,76],[80,49],[56,27],[79,71],[85,76],[61,36],[44,74],[24,57],[20,76],[80,46],[50,59],[60,42],[30,50],[74,5],[61,51],[94,39],[36,96],[68,33],[90,85],[26,22],[68,96],[21,7],[55,67],[89,2],[39,5],[95,50],[26,17],[24,79],[9,54],[9,81],[23,7],[0,68],[79,40],[93,21],[47,64],[30,43],[30,57],[94,47],[98,46],[93,46],[52,70],[17,88],[39,98],[5,82],[80,5],[24,70],[78,97],[42,36]]",
    "[[3,0],[4,0],[5,0],[6,1],[7,2],[7,3],[7,4],[6,5],[5,5],[4,5],[3,5],[2,5],[1,4],[1,3],[1,1],[2,1],[4,2],[4,6]]"
};
var outputs = new string[]
{
    "[[94,10],[19,10],[25,10],[45,10],[99,34],[91,10],[99,41],[30,10],[97,49],[59,10],[87,49],[99,49],[43,10],[58,49],[18,10],[10,30],[82,49],[99,27],[99,12],[37,49],[61,49],[11,11],[81,49],[83,10],[99,21],[10,46],[51,10],[63,49],[58,10],[19,49],[50,10],[74,10],[18,49],[16,10],[12,47],[83,49],[10,32]]",
    "[[72,99],[98,93],[99,86],[18,0],[99,26],[61,99],[98,2],[66,99],[50,0],[99,6],[19,99],[99,58],[3,99],[30,99],[99,23],[39,0],[0,59],[93,99],[29,0],[0,1],[0,95],[57,99],[41,99],[5,0],[0,46],[0,60],[93,0],[0,85],[45,0],[52,0],[56,0],[99,66],[42,99],[0,18],[99,39],[0,70],[99,48],[0,40],[99,29],[99,65],[0,38],[79,99],[0,28],[1,99],[0,6]]",
    "[[0,0],[0,1],[0,2],[1,0],[2,0],[3,0],[3,1],[3,2],[3,3]]",
    "[[0,0],[0,5],[0,9],[4,0],[4,9],[5,0],[6,0],[7,9],[8,7],[9,1],[9,2],[9,5]]",
    "[[1,1],[2,0],[3,3],[2,4],[4,2]]",
    "[[1,1],[2,0],[3,3],[2,4],[4,2]]",
    "[[4,2],[2,2],[1,2]]",
    "[[3,7],[4,13],[11,10],[4,3],[8,5],[7,13]]",
    "[[0,2],[0,0],[2,0],[0,1],[1,0],[4,3],[3,0]]",
    "[[12,2],[1,75],[39,98],[97,97],[99,94],[28,98],[5,82],[98,46],[89,2],[93,7],[57,0],[0,68],[2,12]]",
    "[[1,1],[1,3],[1,4],[2,5],[3,0],[4,0],[4,6],[5,0],[6,1],[6,5],[7,2],[7,3],[7,4]]"
};

for(int i = 0; i < inputs.Length; i++)
{
    var solution = new Solution();
    var trees = System.Text.Json.JsonSerializer.Deserialize<int[][]>(inputs[i]);
    var res = solution.OuterTrees(trees).OrderBy(x => (x[0] << 16) | x[1]).ToList();
    var expected = System.Text.Json.JsonSerializer.Deserialize<List<int[]>>(outputs[i]).OrderBy(x => (x[0] << 16) | x[1]).ToList();
    Console.WriteLine(string.Join(",", res.Select(x => $"[{x[0]},{x[1]}]")));
    Console.WriteLine(string.Join(",", expected.Select(x => $"[{x[0]},{x[1]}]")));
    while(res.Count > 0 && expected.Count > 0)
    {
        bool found = false;
        for(int j = 0; j < expected.Count; j++)
        {
            for(int k = 0; k < res.Count; k++)
            {
                if(expected[j][0] == res[k][0] && expected[j][1] == res[k][1])
                {
                    expected.RemoveAt(j);
                    res.RemoveAt(k);
                    found = true;
                    break;
                }
            }
            if(found)
                break;
        }
        if(!found)
            break;
    }
    if(res.Count > 0 || expected.Count > 0)
        Console.WriteLine("[TEST FAILED]");
    Console.WriteLine();
}

[10,30],[10,32],[10,46],[11,11],[12,47],[16,10],[18,10],[18,49],[19,10],[19,49],[25,10],[30,10],[37,49],[43,10],[45,10],[50,10],[51,10],[58,10],[58,49],[59,10],[61,49],[63,49],[74,10],[81,49],[82,49],[83,10],[83,49],[87,49],[91,10],[94,10],[97,49],[99,12],[99,21],[99,27],[99,34],[99,41],[99,49]
[10,30],[10,32],[10,46],[11,11],[12,47],[16,10],[18,10],[18,49],[19,10],[19,49],[25,10],[30,10],[37,49],[43,10],[45,10],[50,10],[51,10],[58,10],[58,49],[59,10],[61,49],[63,49],[74,10],[81,49],[82,49],[83,10],[83,49],[87,49],[91,10],[94,10],[97,49],[99,12],[99,21],[99,27],[99,34],[99,41],[99,49]

[0,1],[0,6],[0,18],[0,28],[0,38],[0,40],[0,46],[0,59],[0,60],[0,70],[0,85],[0,95],[1,99],[3,99],[5,0],[18,0],[19,99],[29,0],[30,99],[39,0],[41,99],[42,99],[45,0],[50,0],[52,0],[56,0],[57,99],[61,99],[66,99],[72,99],[79,99],[93,0],[93,99],[98,2],[98,93],[99,6],[99,23],[99,26],[99,29],[99,39],[99,48],[99,58],[99,65],[99,66],[99,86]
[0,1],[0,6],[0,18],[0,28],[0,38],[0,40],[0,46],[0,59],[0,60],[0,70],[0,85],